#### Tuesday, November 5, 2024

mamba activate langchain-chroma

[Build a Local RAG Application](https://python.langchain.com/docs/tutorials/local_rag/)

This notebook uses Ollama to serve up Llama 3.1 8b model.

This all runs in one pass.

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [3]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [4]:
docs[0]

Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.')

In [5]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    # model="llama3.1:8b",
       model="llama3.1:latest",
)

In [6]:
response_message = model.invoke(
    "Simulate a rap battle between Stephen Colbert and John Oliver"
)

print(response_message.content)

**The scene is set: a dark, crowded comedy club. The crowd is rowdy and ready for the ultimate showdown in satirical wit. In the blue corner, we have Stephen Colbert, aka "The O'Reilly Factor's" own "Truthy" Truth-teller. In the red corner, John Oliver, the lovable and scathing host of Last Week Tonight. The battle begins!**

**Round 1: Stephen Colbert**

(Stephen takes the stage, adjusting his tie)

Yo, I'm Stephen Colbert, and I'm here to say
My facts are straight, in a world gone astray
I report on the news, with a grin so wide
While you're over here, sipping tea, and letting satire slide

(My apologies, John, but your show's just not as bright
As mine, where we fact-check, day and night)

**Round 1: John Oliver**

(John storms onto the stage, mic in hand)

Hold up, Stephen, let me set you straight
Your 'facts' are cherry-picked, with a partisan weight
You're a master of spin, but I'm not impressed
My show's got substance, and your facts are depressed

(I fact-check too, with a fire

#### Using in a chain

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

'The main themes in these retrieved documents are:\n\n1. **Task Decomposition**: Breaking down complex tasks into smaller, manageable subgoals using:\n\t* Large Language Models (LLMs) with simple prompting\n\t* Task-specific instructions\n\t* Human inputs\n2. **Planning and Execution**: A system for autonomous agents that involves:\n\t* Planning: decomposing tasks into subgoals and planning ahead\n\t* Task execution: expert models executing specific tasks and logging results\n3. **Autonomous Agent System**: A system overview (Fig. 1) that includes components such as:\n\t* Component One: Planning, which involves task decomposition and reflection/refinement\n\nThese themes are related to the development of autonomous agents that can perform complex tasks efficiently and learn from their experiences.'

#### Q&A

In [8]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

'There are three approaches to task decomposition: (1) using simple prompting in a Large Language Model (LLM), (2) employing task-specific instructions, and (3) utilizing human inputs. This breakdown enables efficient handling of complex tasks by breaking them down into smaller, manageable subgoals. It facilitates planning ahead and improves the quality of final results through self-reflection and refinement.'

#### Q&A with retrieval

In [9]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [10]:
question = "What are the approaches to Task Decomposition?"

qa_chain.invoke(question)

'Task decomposition can be done in three ways: (1) by Large Language Model (LLM) with simple prompting, (2) using task-specific instructions, and (3) with human inputs. This process breaks down large tasks into smaller, manageable subgoals for efficient handling of complex tasks. It enables an agent to plan ahead and handle tasks more effectively.'